<a href="https://colab.research.google.com/github/Minyst/ML_DL_Portfolio_KR/blob/main/YOLO/YOLOv10_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!curl -L "https://universe.roboflow.com/ds/rgovMIXJCK?key=fyU64pv91e" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

In [ ]:
!pip install -q supervision roboflow

In [5]:
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt

In [ ]:
%cd {HOME}

!yolo task=detect mode=predict conf=0.25 save=True \
model={HOME}/weights/yolov10n.pt

In [ ]:
!pip install ultralytics

In [8]:
from ultralytics import YOLOv10
import supervision as sv

In [9]:
import cv2
import numpy as np
from tqdm import tqdm
from google.colab.patches import cv2_imshow

In [10]:
output_video_path = '/content/PREMIER LEAGUE HIGHLIGHTS 10sec.mp4'

In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLOv10
model = YOLOv10(f'{HOME}/weights/yolov10n.pt')
model.train(data='/content/data.yaml', epochs=100, imgsz=640)

In [12]:
model = YOLOv10('/content/runs/detect/train/weights/best.pt')

In [13]:
video_path = '/content/PREMIER LEAGUE HIGHLIGHTS 10sec.mp4'
frames = []
cap = cv2.VideoCapture(video_path)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        frames.append(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    else:
        break

In [ ]:
cnt = 0
frame_lst = []
for frame in tqdm(frames):
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    annotated_image = bounding_box_annotator.annotate(
        scene=frame, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections)
    final_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
    frame_lst.append(final_image)
    cnt += 1

In [ ]:
width = final_image.shape[1]
height = final_image.shape[0]
fps = cnt/10
print(width, height, fps)

In [18]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))

for final_image in frame_lst:
    out.write(final_image)
out.release()